In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import PorterStemmer,WordNetLemmatizer
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import keras
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
import spacy
from nltk.stem.porter import *

In [ ]:
true_set = pd.read_csv("dataset/True.csv")
false_set = pd.read_csv("dataset/Fake.csv")

In [ ]:
true_set.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [ ]:
false_set.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [ ]:
true_set['label'] = 1
false_set['label'] = 0

In [ ]:
data = pd.concat([true_set,false_set])
data.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [ ]:
data.isna().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

In [ ]:
data.title.count()

44898

In [ ]:
data.subject.value_counts()

politicsNews       11272
worldnews          10145
News                9050
politics            6841
left-news           4459
Government News     1570
US_News              783
Middle-east          778
Name: subject, dtype: int64

# Data Processing

In [ ]:
data['text'] = data['text'] + " " + data['title'] + " " + data['subject']
del data['title']
del data['subject']
del data['date']
data.head()

,text,label
0,WASHINGTON (Reuters) - The head of a conservat...,1
1,WASHINGTON (Reuters) - Transgender people will...,1
2,WASHINGTON (Reuters) - The special counsel inv...,1
3,WASHINGTON (Reuters) - Trump campaign adviser ...,1
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,1


In [ ]:
# punctuation
nltk.download('stopwords')
stop = set(stopwords.words('english'))
punc = list(string.punctuation)
stop.update(punc)
stop

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'need

In [ ]:
# Perform Stemming
nlp = spacy.load('en_core_web_sm')
stemmer = PorterStemmer()
def stem_text(text):
    doc = nlp(text)
    final_text = []

    for token in doc:
        # If the token is not a stop word and not a recognized entity, stem it
        if token.text not in stop and token.ent_type_ == 0:
            word = stemmer.stem(token.text)
            final_text.append(word)
        else:
            # If it's an entity or a stop word, add it as it is
            final_text.append(token.text)

    return " ".join(final_text)

In [ ]:
data.text = data.text.apply(stem_text)

In [ ]:
plt.figure(figsize = (20,20))
wc = WordCloud(max_words = 3000 , width = 1600 , height = 800 ,
               colormap='winter',
               stopwords = STOPWORDS).generate(" ".join(data.text))
plt.imshow(wc , interpolation = 'bilinear')

In [ ]:
data.text

0        washington (reuters) head conserv republican f...
1        washington (reuters) transgend peopl allow fir...
2        washington (reuters) special counsel investig ...
3        washington (reuters) trump campaign advis geor...
4        seattle/washington (reuters) presid donald tru...
                               ...                        
23476    21st centuri wire say 21wire report earlier we...
23477    21st centuri wire say familiar theme. whenev d...
23478    patrick henningsen 21st centuri wirerememb oba...
23479    21st centuri wire say al jazeera america go hi...
23480    21st centuri wire say 21wire predict new year ...
Name: text, Length: 44898, dtype: object

In [ ]:
# Split data into training and temp (test + validation) sets
X_train, X_temp, y_train, y_temp = train_test_split(data.text, data.label,
                                                    test_size=0.4, random_state=42) # 60% for training

# Split temp into test and validation sets
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp,
                                                test_size=0.5, random_state=42) # 20% for validation, 20% for test

In [ ]:
cv=CountVectorizer(min_df=0,max_df=1,ngram_range=(1,2))
#transformed train reviews
cv_train_reviews=cv.fit_transform(X_train)
#transformed val reviews
cv_val_reviews=cv.fit_transform(X_val)
#transformed test reviews
cv_test_reviews=cv.transform(X_test)

print('BOW_cv_train:',cv_train_reviews.shape)
print('BOW_cv_val:',cv_val_reviews.shape)
print('BOW_cv_test:',cv_test_reviews.shape)

BOW_cv_train: (26938, 1777791)
BOW_cv_val: (8980, 945576)
BOW_cv_test: (8980, 945576)


In [ ]:
print(type(X_train))

<class 'pandas.core.series.Series'>


# Baseline Model

In [96]:
# filter for those that are fake news
fake_news_texts = X_train[y_train == 0]

In [97]:
# Fit the vectorizer on the fake news texts
cv_fake_news = cv.fit_transform(fake_news_texts)

# Sum word occurrences to find frequencies
sum_words = cv_fake_news.sum(axis=0)
words_freq = [(word, sum_words[0, idx]) for word, idx in cv.vocabulary_.items()]
words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)

# Extract top N words as keywords
N = 50  # Adjust N based on your needs
keywords = [word for word, freq in words_freq[:N]]

In [98]:
keywords

['amp amp',
 'quot quot',
 'nbk',
 'quot amp',
 'jonben',
 'amp quot',
 'offense conspiraci',
 'gse',
 'estim undocu',
 'undocu population',
 'push quot',
 'altnatparkser',
 'altusnatparkservic altnatparkser',
 'altnatparkser januari',
 'ar vr',
 'shumpert',
 'roguepotusstaff januari',
 'huber',
 'mickey mallori',
 'quot var',
 'alabedbana',
 'stratton',
 'indexof',
 'nffa',
 'boycottbudwis',
 'petrov',
 'rof',
 'youngblood',
 '39 star',
 'chalabi',
 'rncwithberni',
 'prototype',
 'function return',
 'function if',
 'lampkin',
 'offense possess',
 'rncwithberni berni',
 'osd_listener',
 'quot osd_listener',
 'knox',
 'river blood',
 'if quot',
 'return quot',
 'ohchr',
 'aftertrumpimplod',
 'roan',
 'chaffetztownhal',
 'rolandsmartin',
 'mostyn',
 'morrow']

In [99]:
def keyword_occurrences(text, keywords):
    # Count occurrences of each keyword in the text
    return sum(text.count(keyword) for keyword in keywords)

# Apply this function to your datasets
X_train_keyword_counts = X_train.apply(lambda x: keyword_occurrences(x, keywords))
X_val_keyword_counts = X_val.apply(lambda x: keyword_occurrences(x, keywords))
X_test_keyword_counts = X_test.apply(lambda x: keyword_occurrences(x, keywords))

In [100]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Reshape for sklearn compatibility
X_train_counts = X_train_keyword_counts.values.reshape(-1, 1)
X_test_counts = X_test_keyword_counts.values.reshape(-1, 1)

# Initialize and train the classifier
clf = LogisticRegression(random_state=42)
clf.fit(X_train_counts, y_train)

# Predict on testing set
predictions_test = clf.predict(X_test_counts)

# Evaluate the model
print("Testing Accuracy:", accuracy_score(y_val, predictions_test))
print("Classification Report:\n", classification_report(y_val, predictions_test))


Testing Accuracy: 0.523162583518931
Classification Report:
               precision    recall  f1-score   support

           0       0.52      1.00      0.69      4698
           1       0.00      0.00      0.00      4282

    accuracy                           0.52      8980
   macro avg       0.26      0.50      0.34      8980
weighted avg       0.27      0.52      0.36      8980



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
